(PERFBLAS)=

# 5.2 Herramientas de lenguajes de programación y del sistema operativo para perfilamiento e implementaciones de BLAS

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* 

```

En esta nota revisamos algunas herramientas de los lenguajes de programación y de los sistemas GNU/Linux para perfilamiento de código. También se revisan las operaciones de BLAS en sus diferentes niveles con ejemplos de paquetes en los lenguajes. 

Se presentan códigos y sus ejecuciones en una máquina `m5.2xlarge` de la nube de [AWS](https://aws.amazon.com/). Se sugiere se utilice la AMI --aquí colocar nombre de AMI-- de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con los siguientes *statements*:


```bash

#!/bin/bash
#Virginia region
DEBIAN_FRONTEND=noninteractive
DEB_BUILD_DEPS="build-essential python3-dev python3-pip python3-setuptools software-properties-common libgit2-dev dirmngr libgmp3-dev libmpfr-dev"
#Install OpenBLAS and other tools
DEB_PACKAGES="sudo nano less time git nodejs curl wget htop gfortran linux-tools-$(uname -r) linux-tools-generic libopenblas-dev"
PIP_PACKAGES="line_profiler memory_profiler psutil guppy3 jedi==0.17.2 awscli"
R_KEY="E298A3A825C0D65DFD57CBB651716619E084DAB9"
R_DEB_BUILD_DEPS="focal-cran40 r-base libssl-dev libxml2-dev libcurl4-openssl-dev"
USER=ubuntu
JUPYTERLAB_VERSION=3.0.0
LANG=C.UTF-8
LC_ALL=C.UTF-8
R_SITE_LIBRARY="/usr/local/lib/R/site-library"
R_PACKAGES="\"repr IRdisplay evaluate crayon pbdZMQ devtools uuid digest CVXR tidyverse tictoc microbenchmark\""

apt-get update && export $DEBIAN_FRONTEND && apt-get install -y tzdata

apt-get update && apt-get install -y $DEB_BUILD_DEPS $DEB_PACKAGES && pip3 install --upgrade pip

apt-key adv --keyserver keyserver.ubuntu.com --recv-keys $R_KEY && \
add-apt-repository "deb https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/" && \
apt-get update && \
apt-get install -yt $R_DEB_BUILD_DEPS

#install next packages for ubuntu user

sudo -H --preserve-env -u $USER bash << EOF
export PATH=/home/$USER/.local/bin:$PATH

pip3 install --user jupyter jupyterlab==$JUPYTERLAB_VERSION

jupyter notebook --generate-config && sed -i "s/# c.NotebookApp.password = .*/c.NotebookApp.password = u'sha1:115e429a919f:21911277af52f3e7a8b59380804140d9ef3e2380'/" /home/$USER/.jupyter/jupyter_notebook_config.py

#install numpy and scipy using OpenBLAS installation

pip3 install --user numpy --no-binary numpy
pip3 install --user scipy --no-binary scipy

pip3 install --user $PIP_PACKAGES

#c kernel in jupyter

pip3 install --user git+https://github.com/brendan-rius/jupyter-c-kernel.git && python3 /home/$USER/.local/lib/python3.8/site-packages/jupyter_c_kernel/install_c_kernel --prefix=/home/$USER/.local/

#r kernel in jupyter
sudo chmod gou+wrx $R_SITE_LIBRARY
R -e 'install.packages(strsplit($R_PACKAGES, " ")[[1]], lib="/usr/local/lib/R/site-library/")' && \
R -e 'devtools::install_github("IRkernel/IRkernel")' && \
R -e 'IRkernel::installspec()' #or:R -e 'IRkernel::installspec(user=FALSE)'

#julia kernel in jupyter

sudo mkdir /usr/local/julia-1.6.0
cd
wget https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz
tar zxvf julia-1.6.0-linux-x86_64.tar.gz

sudo cp -r julia-1.6.0/* /usr/local/julia-1.6.0/ 

/usr/local/julia-1.6.0/bin/julia -e 'using Pkg;Pkg.add("IJulia")' && \
/usr/local/julia-1.6.0/bin/julia -e 'using Pkg;Pkg.add(Pkg.PackageSpec(name="JuMP", rev="master"))' && \
/usr/local/julia-1.6.0/bin/julia -e 'using Pkg;Pkg.add("ECOS");Pkg.add("OSQP");Pkg.add("SCS");Pkg.add("GLPK");Pkg.add("Optim")'

#launch jupyter lab

/home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &

EOF

```

Y la máquina `m5.2xlarge` tiene las siguientes características:

```python
%%bash
lscpu
```

## Perfilamiento: medición de tiempo en *Python*

### Módulo: [time](https://docs.python.org/3/library/time.html#time.time)

Imports y funciones de apoyo:

In [2]:
import math
import time

import numpy as np
from pytest import approx
from scipy.integrate import quad


In [3]:
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


Objetivo:

In [4]:
f=lambda x: math.exp(-x**2) #using math library
a=0
b=1
obj, err = quad(f, a, b)
print(obj)

0.7468241328124271


Medición de tiempo:

In [5]:
n=10**2
start_time = time.time()
res=Rcf(f,a,b,n)
end_time = time.time()
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 4.506111145019531e-05 segundos


Prueba que se resuelve correctamente el problema:

In [6]:
print(res == approx(obj))


False


In [8]:
print("hello world")

hello world


### Comando de *magic*: [%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)

```{margin}

Los comandos de *magic* los podemos utilizar en los *jupyter notebooks* pero también con *import*'s.

```

Nos regresa las mediciones siguientes:

* `CPU times` que contiene:

    * `user`: mide la cantidad de tiempo de los *statements* que la CPU gastó para funciones que no están relacionadas con el *kernel* del sistema.
    
    * `sys`: mide la cantidad de tiempo de los *statements* que la CPU gastó en funciones a nivel de kernel del sistema.
    
    * `total`: suma entre el `user` y `sys` para todos **todos los *cores***. 
    
* `Wall time`: mide el *wall clock* o *elapsed time* que se refiere al tiempo desde que inicia la ejecución de los *statements* hasta su finalización.

* `Out`: resultado.



```{admonition} Comentarios

* Para mediciones de tiempos que involucran cómputo en paralelo es posible que `total` exceda a `Wall time` pues es la suma de tiempos para todos los *cores*.

* Recuérdese que algunos ejemplos de funciones relacionadas con el *kernel* del sistema es el alojamiento, lectura y escritura de variables en memoria, las relacionadas con el I/O de disco o *network*.

* La diferencia entre `total` y `Wall time` da una idea de la cantidad de tiempo que se ocupó el sistema en la ejecución de *statements* que no involucran `user` ni a `sys` (por ejemplo un *statement* del tipo `sleep`) o en tareas no relacionadas con los *statements* medidos (por ejemplo si en el momento de la medición se estaban corriendo otros procesos).

```

Al medir tiempos de ejecución, **siempre** hay variación en la medición por lo que se recomienda realizar las mediciones un número repetido de veces. Para este caso el comando de *magic* `%timeit` nos ayuda. 

### Comando de *magic*: [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)

A continuación se mide el tiempo de ejecución para la función `Rcf`. Se promedian los tiempos de $n=5$ ejecuciones y se calcula su desviación estándar. Se reptite lo anterior $r=10$ veces y se reporta el mejor resultado. 

```python
%timeit -n 5 -r 10 Rcf(f,0,1,n)
```

```{admonition} Observación
:class: tip

$ms$ es milisecond, $\mu s$ es microsecond y $ns$ es nanosecond.

```

```{admonition} Comentarios

* `%timeit` se recomienda usar para secciones de código pequeñas. Si se observa una variación en las mediciones de tiempo entre distintas repeticiones entonces hay que realizar más repeticiones hasta tener un resultado estable.

* `%timeit` desabilita temporalmente el *garbage collector* de *Python* (esto es, no habrá desalojamiento en memoria de objetos de Python que no se utilicen). Si el *garbage collector* es invocado en tus *statements*, esto puede ser una razón de posibles diferencias que se obtengan en las mediciones de tiempo.


```

## Perfilamiento: medición de uso de CPU en *Python*

### [line_profiler](https://pypi.org/project/line-profiler/)

### [CProfile](https://docs.python.org/2/library/profile.html) 

*CProfile* es `built-in` en la *standard-library* de Python.

## Perfilamiento: medición de uso de memoria en *Python*

### [memory_profiler](https://pypi.org/project/memory-profiler/)

### [heapy](https://pypi.org/project/guppy/)

## BLAS

### OpenBLAS y *NumPy*

### OpenBLAS y librerías de cómputo matricial de *R*

## Perfilamiento: medición de tiempo en el sistema operativo GNU/Linux

### [/usr/bin/time](https://en.wikipedia.org/wiki/Time_(Unix))


## Perfilamiento: medición de uso de CPU en el sistema operativo GNU/Linux

### [perf](https://github.com/torvalds/linux/tree/master/tools/perf)

```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

